In [ ]:
# default_exp core

# Public data food analysis

> Potential description.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import os
import matplotlib.pyplot as plt
import pickle
from datetime import date 
from datetime import datetime

In [ ]:
#export
def load_public_data(in_path, out_path = 'data/public.pickle'):
    """
    load public data and output processed data in pickle format
    in_path : input path
    out_path: output path
    """
    public_all = pd.read_csv(in_path).drop(columns = ['foodimage_file_name'])
    
    def handle_public_time(s):
        tmp_s = s.replace('p.m.', '').replace('a.m.', '')
        try:
            return pd.to_datetime(' '.join(tmp_s.split()[:2]) )
        except:
            try:
                if int(tmp_s.split()[1][:2]) > 12:
                    tmp_s = s.replace('p.m.', '').replace('a.m.', '').replace('PM', '').replace('pm', '')
                return pd.to_datetime(' '.join(tmp_s.split()[:2]) )
            except:
                return np.nan
        
    original_logtime_notz_lst = []
    for t in (public_all.original_logtime.values):
        original_logtime_notz_lst.append(handle_public_time(t)) 
    public_all['original_logtime_notz'] = original_logtime_notz_lst
    
    public_all = public_all.dropna().reset_index(drop = True)
    
    def find_date(d):
        if d.hour < 4:
            return d.date() - pd.Timedelta('1 day')
        else:
            return d.date()
    public_all['day'] = public_all['original_logtime_notz'].apply(find_date)
    
    
    # Handle the time - Time in floating point format
    public_all['local_time'] = public_all.original_logtime_notz.apply(lambda x: pd.Timedelta(x.time().isoformat()).total_seconds() /3600.).values
    day_begins_at = 4
    public_all.loc[(public_all['local_time'] < day_begins_at), 'local_time'] = 24.0 + public_all.loc[(public_all['local_time'] < day_begins_at), 'local_time']
    
    # Handle the time - Time in Datetime object format
    public_all['time'] = pd.DatetimeIndex(public_all.original_logtime_notz).time
    
    # Handle week from start
    public_start_time_dic = dict(public_all.groupby('unique_code').agg(np.min)['day'])
    def count_week_public(s):
        return (s.day - public_start_time_dic[s.unique_code]).days // 7 + 1
    public_all['week_from_start'] = public_all.apply(count_week_public, axis = 1)
    
    public_all['year'] = public_all.day.apply(lambda d: d.year)
    
    public_all_pickle_file = open(out_path, 'wb')
    pickle.dump(public_all, public_all_pickle_file)     
    print('data is saved at {}'.format(out_path))
    public_all_pickle_file.close() 

In [ ]:
load_public_data('data/test_food_details.csv')

data is saved at data/public.pickle
